# Symbolic Hamiltonian and Transformations

In this notebook, I will show how to systematically construct the symbolic Hamiltonian of fermionic and spin models. I will introduce the Jordan-Wigner transformation and Majorana fermions. The code implementation is based on the open source frameworks, [OpenFermion](https://quantumai.google/openfermion) and [SymPy](https://www.sympy.org/en/index.html). 

## Function Definition

In [25]:
from openfermion import *
from sympy import *


def spin_latex(H_spin_dict):
    """Convert the spin operators into LaTeX expression via SymPy

    Parameters
    ----------
    H_spin_dict : dict
        contains spin operators and their coefficients

    Returns
    -------
    LaTeX expression
        output the symbolic expression in LaTeX form
    """
    expr = []
    for key, value in H_spin_dict.items():
        coefficient = nsimplify(simplify(value))
        if isinstance(coefficient, Integer):
            coefficient = sign(
                coefficient)*UnevaluatedExpr(factorint(abs(coefficient), visual=True))
        operator = []
        for i_key in key:
            operator.append(
                symbols(i_key[1]+'_'+str(i_key[0]), commutative=False))
        expr.append(coefficient*prod(operator))
    return sum(expr)


def fermion_latex(H_fermion_dict):
    """Convert the fermionic operators into LaTeX expression via SymPy

    Parameters
    ----------
    H_fermion_dict : dict
        contains fermionic operators and their coefficients

    Returns
    -------
    LaTeX expression
        output the symbolic expression in LaTeX form
    """
    expr = []
    for key, value in H_fermion_dict.items():
        coefficient = nsimplify(simplify(value))
        if isinstance(coefficient, Integer):
            coefficient = sign(
                coefficient)*UnevaluatedExpr(factorint(abs(coefficient), visual=True))
        if key != ():
            operator = []
            for i_key in key:
                if i_key[1] == 1:
                    operator.append(
                        symbols('c^\dagger_'+str(i_key[0]), commutative=False))
                else:
                    operator.append(
                        symbols('c_'+str(i_key[0]), commutative=False))
        else:
            operator = [1]
        expr.append(coefficient*prod(operator))
    return sum(expr)


def majorana_latex(H_majorana_dict):
    """Convert the Majorana operators into LaTeX expression via SymPy

    Parameters
    ----------
    H_majorana_dict : dict
        contains Majorana operators and their coefficients

    Returns
    -------
    LaTeX expression
        output the symbolic expression in LaTeX form
    """
    H_majorana_dict_nonzero = {key: value for key,
                               value in H_majorana_dict.items() if value != 0}
    expr = []
    for key, value in H_majorana_dict_nonzero.items():
        coefficient = nsimplify(simplify(value))
        if isinstance(coefficient/I, Integer):
            coefficient = sign(
                coefficient)*UnevaluatedExpr(factorint(abs(coefficient), visual=True))
        if key != ():
            operator = []
            for i_key in key:
                operator.append(
                    symbols('\gamma_'+str(i_key), commutative=False))
        else:
            operator = [1]
        expr.append(coefficient*prod(operator))
    return sum(expr)


## Spin Chain

The general anisotropic 1D spin-1/2 chain of 4 sites with nearest-neighbor exchange couplings under the open boundary condition is described by the Hamiltonian in Equation (1.12).

In [26]:
n_qubits = 4
alpha, beta_, g = symbols('alpha, beta, g')
H_spin = QubitOperator()
for j in range(n_qubits-1):
    H_spin += QubitOperator(((j, 'Z')), -Indexed(g, j))
    H_spin += QubitOperator(((j, 'X'), (j+1, 'X')), -1)
    H_spin += QubitOperator(((j, 'Y'), (j+1, 'Y')), -Indexed(alpha, j))
    H_spin += QubitOperator(((j, 'Z'), (j+1, 'Z')), -Indexed(beta_, j))
H_spin += QubitOperator(((n_qubits-1, 'Z')), -Indexed(g, n_qubits-1))
spin_latex(H_spin.terms)


-alpha[0]*Y_0*Y_1 - alpha[1]*Y_1*Y_2 - alpha[2]*Y_2*Y_3 - beta[0]*Z_0*Z_1 - beta[1]*Z_1*Z_2 - beta[2]*Z_2*Z_3 - g[0]*Z_0 - g[1]*Z_1 - g[2]*Z_2 - g[3]*Z_3 - 1*X_0*X_1 - 1*X_1*X_2 - 1*X_2*X_3

## The $p$-wave superconductor

The one-dimensional tight-binding $p$-wave superconductor, whose Hamiltonian in Equation (1.16) is quadratic.

In [27]:
n_sites = 4
t, Delta, mu = symbols('t, Delta, mu')
H_fermion = FermionOperator()
for j in range(n_sites-1):
    H_fermion += FermionOperator(((j, 1), (j, 0)), -
                                 Indexed(mu, j)) + Rational(1/2)*Indexed(mu, j)
    H_fermion += FermionOperator(((j, 1), (j+1, 0)), -Indexed(t, j))
    H_fermion += FermionOperator(((j+1, 1), (j, 0)), -
                                 Indexed(t, j).conjugate())
    H_fermion += FermionOperator(((j, 1), (j+1, 1)), -Indexed(Delta, j))
    H_fermion += FermionOperator(((j+1, 0), (j, 0)), -
                                 Indexed(Delta, j).conjugate())
H_fermion += FermionOperator(((n_sites-1, 1), (n_sites-1, 0)), -
                             Indexed(mu, n_sites-1)) + Rational(1/2)*Indexed(mu, n_sites-1)
fermion_latex(H_fermion.terms)


-conjugate(Delta[0])*c_1*c_0 - conjugate(Delta[1])*c_2*c_1 - conjugate(Delta[2])*c_3*c_2 - conjugate(t[0])*c^\dagger_1*c_0 - conjugate(t[1])*c^\dagger_2*c_1 - conjugate(t[2])*c^\dagger_3*c_2 - Delta[0]*c^\dagger_0*c^\dagger_1 - Delta[1]*c^\dagger_1*c^\dagger_2 - Delta[2]*c^\dagger_2*c^\dagger_3 + mu[0]/2 - mu[0]*c^\dagger_0*c_0 + mu[1]/2 - mu[1]*c^\dagger_1*c_1 + mu[2]/2 - mu[2]*c^\dagger_2*c_2 + mu[3]/2 - mu[3]*c^\dagger_3*c_3 - t[0]*c^\dagger_0*c_1 - t[1]*c^\dagger_1*c_2 - t[2]*c^\dagger_2*c_3

We assign some numerical value for the coupling parameters

In [28]:
n_sites = 4
t, Delta, mu = 3, 5, 7
H_fermion = FermionOperator()
for j in range(n_sites-1):
    H_fermion += FermionOperator(((j, 1), (j, 0)), -mu) + 1/2*mu
    H_fermion += FermionOperator(((j, 1), (j+1, 0)), -t)
    H_fermion += FermionOperator(((j+1, 1), (j, 0)), -t)
    H_fermion += FermionOperator(((j, 1), (j+1, 1)), -Delta)
    H_fermion += FermionOperator(((j+1, 0), (j, 0)), -Delta)
H_fermion += FermionOperator(((n_sites-1, 1), (n_sites-1, 0)), -mu) + 1/2*mu
fermion_latex(H_fermion.terms).doit()


14 - 5*c^\dagger_0*c^\dagger_1 - 7*c^\dagger_0*c_0 - 3*c^\dagger_0*c_1 - 5*c^\dagger_1*c^\dagger_2 - 3*c^\dagger_1*c_0 - 7*c^\dagger_1*c_1 - 3*c^\dagger_1*c_2 - 5*c^\dagger_2*c^\dagger_3 - 3*c^\dagger_2*c_1 - 7*c^\dagger_2*c_2 - 3*c^\dagger_2*c_3 - 3*c^\dagger_3*c_2 - 7*c^\dagger_3*c_3 - 5*c_1*c_0 - 5*c_2*c_1 - 5*c_3*c_2

The Hermitian part $S = S^\dagger$ of the Bogoliubov-de Gennes Hamiltonian is

In [29]:
get_quadratic_hamiltonian(H_fermion).hermitian_part.real


array([[-7., -3.,  0.,  0.],
       [-3., -7., -3.,  0.],
       [ 0., -3., -7., -3.],
       [ 0.,  0., -3., -7.]])

The anti-symmetric part $A = - A^T$ of the Bogoliubov-de Gennes Hamiltonian is

In [30]:
get_quadratic_hamiltonian(H_fermion).antisymmetric_part.real


array([[ 0., -5.,  0.,  0.],
       [ 5.,  0., -5.,  0.],
       [ 0.,  5.,  0., -5.],
       [ 0.,  0.,  5.,  0.]])

## Majorana Representation

We can transform the Hamiltonian of the $p$-wave superconducting model from fermionic representation to Majorana representation (we set $t_i=t^*_i=\Delta_i=\Delta^*_i=3$ and $\mu_i = 2$ for demonstration):

In [31]:
n_sites = 4
t, Delta, mu = 3, 3, 2
H_fermion = FermionOperator()
for j in range(n_sites-1):
    H_fermion += FermionOperator(((j, 1), (j, 0)), -mu) + 1/2*mu
    H_fermion += FermionOperator(((j, 1), (j+1, 0)), -t)
    H_fermion += FermionOperator(((j+1, 1), (j, 0)), -t)
    H_fermion += FermionOperator(((j, 1), (j+1, 1)), -Delta)
    H_fermion += FermionOperator(((j+1, 0), (j, 0)), -Delta)
H_fermion += FermionOperator(((n_sites-1, 1), (n_sites-1, 0)), -mu) + 1/2*mu
fermion_latex(H_fermion.terms).doit()


4 - 3*c^\dagger_0*c^\dagger_1 - 2*c^\dagger_0*c_0 - 3*c^\dagger_0*c_1 - 3*c^\dagger_1*c^\dagger_2 - 3*c^\dagger_1*c_0 - 2*c^\dagger_1*c_1 - 3*c^\dagger_1*c_2 - 3*c^\dagger_2*c^\dagger_3 - 3*c^\dagger_2*c_1 - 2*c^\dagger_2*c_2 - 3*c^\dagger_2*c_3 - 3*c^\dagger_3*c_2 - 2*c^\dagger_3*c_3 - 3*c_1*c_0 - 3*c_2*c_1 - 3*c_3*c_2

Here is an instance of Equation (1.46).

In [32]:
H_majorana_dict = get_majorana_operator(H_fermion).terms
majorana_latex(H_majorana_dict).simplify()


I*(-\gamma_0*\gamma_1 + 3*\gamma_1*\gamma_2 - \gamma_2*\gamma_3 + 3*\gamma_3*\gamma_4 - \gamma_4*\gamma_5 + 3*\gamma_5*\gamma_6 - \gamma_6*\gamma_7)

## Jordan-Wigner Transformation

Here we explicitly how to transform the spin Hamiltonian in Equation (1.12) into fermionic Hamiltonian in Equation (1.65). We remark that OpenFermion adopts the transformation in the quantum chemistry community, which differs a minus sign of the couplings $g$ along the $z$-direction

In [21]:
n_qubits = 4
alpha, beta_, g = 3, 5, -7
H_spin = QubitOperator()
for j in range(n_qubits-1):
    H_spin += QubitOperator(((j, 'Z')), -g)
    H_spin += QubitOperator(((j, 'X'), (j+1, 'X')), -1)
    H_spin += QubitOperator(((j, 'Y'), (j+1, 'Y')), -alpha)
    H_spin += QubitOperator(((j, 'Z'), (j+1, 'Z')), -beta_)
H_spin += QubitOperator(((n_qubits-1, 'Z')), -g)
spin_latex(H_spin.terms)


-1*X_0*X_1 - 1*X_1*X_2 - 1*X_2*X_3 - 3**1*Y_0*Y_1 - 3**1*Y_1*Y_2 - 3**1*Y_2*Y_3 - 5**1*Z_0*Z_1 - 5**1*Z_1*Z_2 - 5**1*Z_2*Z_3 + 7**1*Z_0 + 7**1*Z_1 + 7**1*Z_2 + 7**1*Z_3

Apply the Jordan-Wigner transformation, one can obtain the fermionic representation.

In [22]:
H_fermion = normal_ordered(reverse_jordan_wigner(H_spin))
fermion_latex(H_fermion.terms)


13**1 - 2**1*c^\dagger_1*c^\dagger_0 - 2**1*c^\dagger_2*c^\dagger_1 - 2**1*c^\dagger_3*c^\dagger_2 + 2**1*c_1*c_0 + 2**1*c_2*c_1 + 2**1*c_3*c_2 - 2**2*c^\dagger_0*c_0 - 2**2*c^\dagger_0*c_1 - 2**2*c^\dagger_1*c_0 - 2**2*c^\dagger_1*c_2 - 2**2*c^\dagger_2*c_1 - 2**2*c^\dagger_2*c_3 - 2**2*c^\dagger_3*c_2 - 2**2*c^\dagger_3*c_3 + (2**1*3**1)*c^\dagger_1*c_1 + (2**1*3**1)*c^\dagger_2*c_2 + (2**2*5**1)*c^\dagger_1*c^\dagger_0*c_1*c_0 + (2**2*5**1)*c^\dagger_2*c^\dagger_1*c_2*c_1 + (2**2*5**1)*c^\dagger_3*c^\dagger_2*c_3*c_2

We can also verify the transformation from Equation (4.1) to Equation (4.2).

In [23]:
n_qubits = 4
J, t, gamma_, delta, g = 1, 7/2, 1/2, 5, -13
H_spin = QubitOperator()
for j in range(n_qubits-1):
    H_spin += QubitOperator(((j, 'Z')), -J*g)
    H_spin += QubitOperator(((j, 'X'), (j+1, 'X')), -J*(t + gamma_))
    H_spin += QubitOperator(((j, 'Y'), (j+1, 'Y')), -J*(t - gamma_))
    H_spin += QubitOperator(((j, 'Z'), (j+1, 'Z')), -J*delta)
H_spin += QubitOperator(((n_qubits-1, 'Z')), -J*g)
spin_latex(H_spin.terms)


13**1*Z_0 + 13**1*Z_1 + 13**1*Z_2 + 13**1*Z_3 - 2**2*X_0*X_1 - 2**2*X_1*X_2 - 2**2*X_2*X_3 - 3**1*Y_0*Y_1 - 3**1*Y_1*Y_2 - 3**1*Y_2*Y_3 - 5**1*Z_0*Z_1 - 5**1*Z_1*Z_2 - 5**1*Z_2*Z_3

In [24]:
H_fermion = normal_ordered(reverse_jordan_wigner(H_spin))
fermion_latex(H_fermion.terms)


1*c^\dagger_1*c^\dagger_0 + 1*c^\dagger_2*c^\dagger_1 + 1*c^\dagger_3*c^\dagger_2 - 1*c_1*c_0 - 1*c_2*c_1 - 1*c_3*c_2 - 2**4*c^\dagger_0*c_0 - 2**4*c^\dagger_3*c_3 + 37**1 - 7**1*c^\dagger_0*c_1 - 7**1*c^\dagger_1*c_0 - 7**1*c^\dagger_1*c_2 - 7**1*c^\dagger_2*c_1 - 7**1*c^\dagger_2*c_3 - 7**1*c^\dagger_3*c_2 - 2**1*3**1*c^\dagger_1*c_1 - 2**1*3**1*c^\dagger_2*c_2 + (2**2*5**1)*c^\dagger_1*c^\dagger_0*c_1*c_0 + (2**2*5**1)*c^\dagger_2*c^\dagger_1*c_2*c_1 + (2**2*5**1)*c^\dagger_3*c^\dagger_2*c_3*c_2